In [2]:
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

In [3]:
from algorithm_utils import *
import general_utils as gu

In [4]:
df: pd.DataFrame = gu.load_local_data().iloc[300:400].reset_index(drop=True)

In [5]:
zigzag_df = zigzag(df)